# My Tools

This notebook contains functions for :

*   Openning dataset
*   Visulaizing image and mask
*   OverLaying Mask on image
*   Distribution of organ (segmented part) in images

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/DataScience/Spleen'
!ls

/content/drive/MyDrive/DataScience/Spleen
NoteBooks  Task09_Spleen  Task09_Spleen.tar


In [3]:
%cd Task09_Spleen/
!ls

/content/drive/MyDrive/DataScience/Spleen/Task09_Spleen
2D_Spleen  dataset.json  imagesTr  imagesTs  labelsTr


In [4]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

## openning dataset

In [5]:
# this function loads the nifti image and returns it
def get_nifti_image(filepath,filename) :
    imgfile = os.path.join(filepath,filename)
    image = nib.load(imgfile)

    return image #image type is 'nibabel.nifti1.Nifti1Image'

In [6]:
# this function prints nifti_image's header. (there are several data about image in header)
def print_nifti_header(nifti_image):
    print(nifti_image.header)

In [7]:
# this function prints nifti_image dim. for example 3D
def print_nifti_image_dim(nifti_image) :
    print(nifti_image.header['dim'][0])

In [8]:
# this function prints the shape of the data. example : for 3D image returns triple
def print_nifti_image_shape(nifti_image) :
    print(nifti_image.header.get_data_shape())

In [9]:
# this function returns a 3D array of a nifti image
def get_data(filepath,filename) : 
    nifti_image = get_nifti_image(filepath,filename)
    data = nifti_image.get_data()

    # Set NAN to 0
    data[np.isnan(data)] = 0

    return data  #data type is 'numpy.ndarray'

In [10]:
# this function returns a 3D array of a nifti image
def get_data_via_file(file) :
    file = os.path.join(file) 
    nifti_image = nib.load(file)
    data = nifti_image.get_data()

    # Set NAN to 0
    data[np.isnan(data)] = 0

    return data  #data type is 'numpy.ndarray'

In [11]:
# this function returns shape of the data
def get_shape(data) :
    return data.shape

In [12]:
def get_slice_count(data) :
    return data.shape[2]

## visualizing image and mask


In [ ]:
datapath = "imagesTr"
dataname = "spleen_19.nii.gz"
maskpath = "labelsTr"
maskname = "spleen_19.nii.gz"

data = get_data(datapath,dataname) # type = np 3D array
mask = get_data(maskpath,maskname) # type = np 3D array

print("data shape : ",get_shape(data))
print("mask shape : ",get_shape(mask))

print("data slice count : ",get_slice_count(data))
print("mask slice count : ",get_slice_count(mask))

In [ ]:
# this function gets 3D np array and slice, then plots it
def plot_data(data,slicenumber) :
    # Plot main window
    fig = plt.figure(
        facecolor='black',
        figsize=(5,5),
        dpi=100
    )

    imgplot = plt.imshow(np.rot90(data[:,:,slicenumber]))
    imgplot.set_cmap('gray')
    plt.axis('off')

In [ ]:
def plot_data_and_mask(data,mask,slicenumber) :
      # Plot main window
      fig = plt.figure(
          #facecolor='black',
          figsize=(15,30),
          dpi=100
      )


      plt.subplot(1, 3, 1)
      imgplot1 =plt.imshow(np.rot90(data[:,:,slicenumber]),cmap='gray')
      
      plt.subplot(1, 3, 2)
      imgplot2 =plt.imshow(np.rot90(mask[:,:,slicenumber]),cmap='gray')

      plt.subplot(1, 3, 3)
      imgplot3 =plt.imshow(np.rot90(data[:,:,slicenumber]),cmap='gray')
      imgplot4 =plt.imshow(np.rot90(mask[:,:,slicenumber]),alpha=0.4,cmap='Reds')

In [ ]:
plot_data_and_mask(data,mask,30)

# Distribution of segmented part

  (not trustable)



In [ ]:
def get_mask_distribution(path,max_length):
  MaskIndice = np.zeros([6,max_length],dtype = int)  
  masklist = os.listdir(path) 
  for file in masklist:    
    name = os.fsdecode(file)
    if name.startswith("spleen"):
      mask = get_data(path,name)
      D = np.where(mask != 0)
      for d in [0,1,2] :
        min = D[d][0]
        max = D[d][-1]
        MaskIndice[2*d][min] += 1
        MaskIndice[(2*d)+1][max] += 1
  return MaskIndice

In [ ]:
D = get_mask_distribution("/content/drive/My Drive/DataScience/Spleen/Task09_Spleen/labelsTr",512)

In [ ]:
def get_accumulated_distribution(D) :
  acc_D = np.copy(D)
  (r,c) = acc_D.shape
  for i in range(0,r) :
    for j in range(1,c) :
        acc_D[i,j] = acc_D[i,j]+ acc_D[i,j-1]
  return acc_D

In [ ]:
acc_D = get_accumulated_distribution(D) 

In [ ]:
# accumulated distribution of min-x == 1
y = acc_D[0,:]
x = np.arange(512)
plt.figure(figsize=(20,8))
plt.axis([20,200,0,50])
plt.bar(x,y,2)
plt.xlabel('x')
plt.ylabel('accumulated min-x = 1')

In [ ]:
# accumulated distribution of max-x == 1
y = acc_D[1,:]
x = np.arange(512)
plt.figure(figsize=(20,8))
plt.axis([120,300,0,50])
plt.bar(x,y,2)
plt.xlabel('x')
plt.ylabel('accumulated max-x = 1')

In [ ]:
# accumulated distribution of min-y == 1
y = acc_D[2,:]
x = np.arange(512)
plt.figure(figsize=(20,8))
plt.axis([150,350,0,50])
plt.bar(x,y,2)
plt.xlabel('y')
plt.ylabel('accumulated min-y = 1')

In [ ]:
# accumulated distribution of max-y == 1
y = acc_D[3,:]
x = np.arange(512)
plt.figure(figsize=(20,8))
plt.axis([150,300,0,50])
plt.bar(x,y,2)
plt.xlabel('y')
plt.ylabel('accumulated max-y = 1')

In [ ]:
# accumulated distribution of min-z == 1
y = acc_D[4,:]
x = np.arange(512)
plt.figure(figsize=(20,8))
plt.axis([0,150,0,50])
plt.bar(x,y,2)
plt.xlabel('z')
plt.ylabel('accumulated min-z = 1')

In [ ]:
# accumulated distribution of max-z == 1
y = acc_D[5,:]
x = np.arange(512)
plt.figure(figsize=(20,8))
plt.axis([0,150,0,50])
plt.bar(x,y,2)
plt.xlabel('z')
plt.ylabel('accumulated max-z = 1')

# boxing masks - finding minimal containing box

In [ ]:
def get_masks(path):
  thelist = os.listdir(path)
  masklist = []
  for mask in thelist :
    name = os.fsdecode(mask)
    if name.startswith("spleen"):
      masklist.append(mask)
  return masklist

In [ ]:
path = "/content/drive/My Drive/DataScience/Spleen/Task09_Spleen/labelsTr"
masklist = get_masks(path)

In [ ]:
# this function returns a matrix wich columns are [min-x,max-x,min-y,max-y,min-z,max-z] of masks
def get_mask_distribution(path,masklist):
  MaskIndice = np.zeros([len(masklist),6],dtype = int)  
  for i in range(len(masklist)):
      name = os.fsdecode(masklist[i])  
      mask = get_data(path,name)
      D = np.where(mask != 0)
      for d in [0,1,2] :
        min = D[d][0]
        max = D[d][-1]
        MaskIndice[i,2*d] = min
        MaskIndice[i,2*d+1] = max
  return MaskIndice

In [ ]:
D = get_mask_distribution(path,masklist)

In [ ]:
def get_box_list(masklist,D):
    l = len(masklist)
    boxlist = []
    for i in range(l):
      name = os.fsdecode(masklist[i])
      name = name.replace(".nii.gz","")
      newlist = [name]
      newlist.extend(D[i])
      boxlist.append(newlist)
    return boxlist

In [ ]:
boxlist = get_box_list(masklist,D)

In [ ]:
from tabulate import tabulate
table = [["mask-name","min-x","max-x","min-y","max-y","min-z","max-z"]]
table.extend(boxlist)
print(tabulate(table, tablefmt='fancy_grid'))

In [ ]:
x_min_total = min(D[:,0])
x_max_total = max(D[:,1])

y_min_total = min(D[:,2])
y_max_total = max(D[:,3])

z_min_total = min(D[:,4])
z_max_total = max(D[:,5])

print([x_min_total,x_max_total,y_min_total,y_max_total,z_min_total,z_max_total])

## conclusion
  organs are located in the box

*   $ x \in [43, 224] $
*   $ y \in [170, 260] $
*   $ z \in [0, 125] $

over the train set


